<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Scraping-abstracts-for-central-bank-research" data-toc-modified-id="Scraping-abstracts-for-central-bank-research-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Scraping abstracts for central bank research</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Function-(1)" data-toc-modified-id="Function-(1)-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Function (1)</a></span></li><li><span><a href="#Function-(2)" data-toc-modified-id="Function-(2)-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Function (2)</a></span></li><li><span><a href="#Function-(3)" data-toc-modified-id="Function-(3)-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Function (3)</a></span><ul class="toc-item"><li><span><a href="#Call-function-(3)" data-toc-modified-id="Call-function-(3)-1.0.3.1"><span class="toc-item-num">1.0.3.1&nbsp;&nbsp;</span>Call function (3)</a></span></li></ul></li><li><span><a href="#Function-(4)" data-toc-modified-id="Function-(4)-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>Function (4)</a></span><ul class="toc-item"><li><span><a href="#Call-function-(4)-using-concurrency-via-threading!" data-toc-modified-id="Call-function-(4)-using-concurrency-via-threading!-1.0.4.1"><span class="toc-item-num">1.0.4.1&nbsp;&nbsp;</span>Call function (4) using concurrency via threading!</a></span></li><li><span><a href="#NB:-Change-the-number-of-threads-if-the-IDEAS-page-throws-a-request-error" data-toc-modified-id="NB:-Change-the-number-of-threads-if-the-IDEAS-page-throws-a-request-error-1.0.4.2"><span class="toc-item-num">1.0.4.2&nbsp;&nbsp;</span>NB: Change the number of threads if the IDEAS page throws a request error</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Scraping abstracts for central bank research

In [1]:
# import sys
# !{sys.executable} -m pip install Scrapy
# !{sys.executable} -m pip install wordcloud
import csv
from bs4 import BeautifulSoup
import requests
from bs4 import NavigableString
import queue
from concurrent.futures import ThreadPoolExecutor
import itertools
import threading
import csv
import pandas as pd 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import numpy as np
import re

### Function (1)
**Return a list with a link to each paper**
* NB: This is for a single page

In [2]:
def get_links(soup):
    
    links = []
    for link in soup.findAll('a', attrs={'href': re.compile("/p/")}):
        links.append("https://ideas.repec.org"+link.get('href'))

    return links 

### Function (2)
**Soup the necessary URL and call Function (1) to return the list of links**

In [3]:
def parse_links(START_URL):
    
    page   = requests.get(START_URL)
    soup   = BeautifulSoup(page.text, 'html.parser')
    links = get_links(soup)

    return links

### Function (3)  
**For a given central bank, go through each page and collect the links by calling Function (2) for each page**

1. use Function (2) to get all the links from the first page for the first central bank
2. access that URL's .html **again**
3. get the links that direct to the next pages i.e. [1],[2]... for that central bank
4. if the last item in that list is not empty get the link to the next page
5. return to 1. above using the link saved in 4.
6. if there and no pages, quit the loop

In [4]:
def parse_all_links(START_URL):
    
    items_list = [] 
    current_link = START_URL   # Setting up the starting link
    
    no_more_pages = False
    while no_more_pages == False:
        
        items_list = items_list + parse_links(current_link)   # Add items from page to the list
        request = requests.get(current_link)                   # Accessing text content again
        parsed_request = BeautifulSoup(request.content, 'html.parser')   # Parsing it again
        link = parsed_request.find_all('li', class_= "page-item")        # Finding next link from within tree
        try:
            if not link[-1].find('a')['href'] is False:
                current_link = current_link.rsplit('/', 1)[0]+'/'+link[-1].find('a')['href']
        except:
            no_more_pages = True
    items_list.pop(0)
    items_list.pop(0)
    return items_list

#### Call function (3)

In [5]:
%%time
# AJUSTE BECS 20251113
links = parse_all_links("https://ideas.repec.org/s/bdm/wpaper.html")         # Banco de México
links.extend(parse_all_links("https://ideas.repec.org/s/rbp/wpaper.html"))   # Banco Central de Reserva del Perú
links.extend(parse_all_links("https://ideas.repec.org/s/chb/bcchwp.html"))   # Banco Central de Chile
links.extend(parse_all_links("https://ideas.repec.org/s/bcr/wpaper.html"))   # Banco Central de Argentina 
links.extend(parse_all_links("https://ideas.repec.org/s/bcb/wpaper.html"))   # Banco Central de Brasil
links.extend(parse_all_links("https://ideas.repec.org/s/bku/doctra.html"))   # Banco Central de Uruguay
links.extend(parse_all_links("https://ideas.repec.org/s/bde/wpaper.html"))   # Banco de España
links.extend(parse_all_links("https://ideas.repec.org/s/bok/wpaper.html"))   # Banco de Corea
links.extend(parse_all_links("https://ideas.repec.org/s/bth/wpaper.html"))   # Banco de Tailandia
links.extend(parse_all_links("https://ideas.repec.org/s/bog/wpaper.html"))   # Banco de Grecia
links.extend(parse_all_links("https://ideas.repec.org/s/bfr/banfra.html"))   # Banco de Francia
links.extend(parse_all_links("https://ideas.repec.org/s/rba/rbardp.html"))   # Banco de Australia
links.extend(parse_all_links("https://ideas.repec.org/s/fip/fedgfe.html"))   # Fed Estados Unidos
links.extend(parse_all_links("https://ideas.repec.org/s/ecb/ecbwps.html"))   # Banco Central Europeo
links.extend(parse_all_links("https://ideas.repec.org/s/boj/bojwps.html"))   # Banco de Japón
links.extend(parse_all_links("https://ideas.repec.org/s/bca/bocawp.html"))   # Banco de Canadá
links.extend(parse_all_links("https://ideas.repec.org/s/nzb/nzbdps.html"))   # Banco de Nueva Zelanda
links.extend(parse_all_links("https://ideas.repec.org/s/boe/boeewp.html"))   # Banco de Inglaterra
links.extend(parse_all_links("https://ideas.repec.org/s/hhs/rbnkwp.html"))   # Banco de Suiza
links.extend(parse_all_links("https://ideas.repec.org/s/bno/worpap.html"))   # Banco de Noruega

Wall time: 2min 19s


### Function (4) 
**For a given abstract page, pull the text of that abstract**

NB: We need the except pass conditions:
* before 1996 for the Fed (here: https://ideas.repec.org/s/fip/fedgfe9.html) the results are no longer interpretable by the program

In [6]:
def get_abstracts(URL):
    
    abstracts = []
    
    page      = requests.get(URL)
    soup      = BeautifulSoup(page.text, 'html.parser')
    
    abstract  = soup.find('div', id='abstract-body').get_text()
    
    try:
        
        info      = [desc.strip() for desc in 
                     soup.find("li",class_="list-group-item downfree").descendants if type(desc) == NavigableString]
        
        abstracts.append({'Author/s': info[0],'Year': info[0].split()[-2], 
                      'Title': info[1], 'ID': info[4], 'Abstract': abstract})
        
    except:
        pass
    
    return abstracts

#### Call function (4) using concurrency via threading!
* Give each thread a different task (URL)
* In this case, we will handle all the tasks at once
* This is great for the purpose at hand, as I have a list of pre-defined URLs to process

#### NB: Change the number of threads if the IDEAS page throws a request error

In [7]:
%%time
# AJUSTE BECS 20251113
bancos = ["https://ideas.repec.org/s/bdm/wpaper.html"         # Banco de México
,"https://ideas.repec.org/s/rbp/wpaper.html"   # Banco Central de Reserva del Perú
,"https://ideas.repec.org/s/chb/bcchwp.html"   # Banco Central de Chile
,"https://ideas.repec.org/s/bcr/wpaper.html"   # Banco Central de Argentina 
,"https://ideas.repec.org/s/bcb/wpaper.html"   # Banco Central de Brasil
,"https://ideas.repec.org/s/bku/doctra.html"   # Banco Central de Uruguay
,"https://ideas.repec.org/s/bde/wpaper.html"   # Banco de España
,"https://ideas.repec.org/s/bok/wpaper.html"   # Banco de Corea
,"https://ideas.repec.org/s/bth/wpaper.html"   # Banco de Tailandia
,"https://ideas.repec.org/s/bog/wpaper.html"   # Banco de Grecia
,"https://ideas.repec.org/s/bfr/banfra.html"   # Banco de Francia
,"https://ideas.repec.org/s/rba/rbardp.html"   # Banco de Australia
,"https://ideas.repec.org/s/fip/fedgfe.html"   # Fed Estados Unidos
#,"https://ideas.repec.org/s/ecb/ecbwps.html"   # Banco Central Europeo
,"https://ideas.repec.org/s/boj/bojwps.html"   # Banco de Japón
,"https://ideas.repec.org/s/bca/bocawp.html"   # Banco de Canadá
,"https://ideas.repec.org/s/nzb/nzbdps.html"   # Banco de Nueva Zelanda
,"https://ideas.repec.org/s/boe/boeewp.html"   # Banco de Inglaterra
#,"https://ideas.repec.org/s/hhs/rbnkwp.html"   # Banco de Suiza
,"https://ideas.repec.org/s/bno/worpap.html"   # Banco de Noruega
, "https://ideas.repec.org/s/bkr/wpaper.html"  # Banco de Rusia
#"https://ideas.repec.org/s/rbz/wpaper.html" 
         ]  # Banco de Sudafrica  

Wall time: 0 ns


In [8]:
names = ['Mexico','Peru', 'Chile', 'Argentina', 'Brasil', 'Uruguay','España','Corea', 'Tailandia',
            'Grecia', 'Francia','Australia','USA',#'BCE',
         'Japon', 'Canada', 'NZelanda', 'Inglaterra',#'Suiza', 
         'Noruega', 'Rusia'#,'Sudafrica'
        ]

In [9]:
%%time
#AJUSTE BECS
for i in range(0, len(bancos)):
    print(names[i]+"|"+str(i)+'/'+str(len(bancos)))
    #print(i)
    links = parse_all_links(bancos[i])  
    with ThreadPoolExecutor(70) as pool:
        abstracts = pool.map(get_abstracts, links)

        abstracts = list(abstracts)
            
    with open('_data_'+names[i]+'.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    #
        if abstracts and abstracts[0]:
            fieldnames = abstracts[0][0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for sublist in abstracts:
                for row in sublist:
                    writer.writerow(row)
    print(names[i]+" finalizado.")

Mexico|0/19
Mexico finalizado.
Peru|1/19
Peru finalizado.
Chile|2/19
Chile finalizado.
Argentina|3/19
Argentina finalizado.
Brasil|4/19
Brasil finalizado.
Uruguay|5/19
Uruguay finalizado.
España|6/19
España finalizado.
Corea|7/19
Corea finalizado.
Tailandia|8/19
Tailandia finalizado.
Grecia|9/19
Grecia finalizado.
Francia|10/19
Francia finalizado.
Australia|11/19
Australia finalizado.
USA|12/19
USA finalizado.
Japon|13/19
Japon finalizado.
Canada|14/19
Canada finalizado.
NZelanda|15/19
NZelanda finalizado.
Inglaterra|16/19
Inglaterra finalizado.
Noruega|17/19
Noruega finalizado.
Rusia|18/19
Rusia finalizado.
Wall time: 30min 50s
